In [1]:
import pandas as pd

In [2]:
full = pd.read_csv('../nuovi notebooks/data/bon/full_preproc_train.csv')

In [3]:
len(full.columns)

36

In [4]:
full.columns

Index(['bot_words_score', 'gen_words_score', 'avg_len', 'max_len', 'min_len',
       'avg_ret', 'max_ret', 'min_ret', 'avg_fav', 'max_fav', 'min_fav',
       'avg_hash', 'max_hash', 'min_hash', 'freq', 'ret_perc', 'media_perc',
       'url_perc', 'quote_perc', 'tweet_intradistance', 'url_entropy',
       'statuses_count', 'followers_count', 'friends_count',
       'favourites_count', 'listed_count', 'url', 'default_profile',
       'profile_use_background_image', 'verified', 'nsfw_profile', 'target',
       'description_len', 'name_len', 'screen_name_len', 'age'],
      dtype='object')

In [5]:
train = pd.read_csv('data/full/train_1000.csv')

In [6]:
train.columns

Index(['Unnamed: 0', 'avg_fav', 'avg_hash', 'avg_len', 'avg_ret',
       'default_profile', 'favourites_count', 'followers_count', 'freq',
       'friends_count', 'listed_count', 'max_fav', 'max_hash', 'max_len',
       'max_ret', 'media_perc', 'min_fav', 'min_hash', 'min_len', 'min_ret',
       'nsfw_avg', 'nsfw_profile', 'profile_use_background_image',
       'quote_perc', 'ret_perc', 'statuses_count', 'target',
       'tweet_intradistance', 'url', 'url_perc', 'user_id', 'description_len',
       'name_len', 'screen_name_len', 'age', 'porn_words_score',
       'prop_words_score', 'spam_words_score', 'fake_words_score',
       'url_entropy'],
      dtype='object')

In [7]:
train = train.drop(columns=['Unnamed: 0', 'nsfw_avg', 'porn_words_score', 'prop_words_score', 'spam_words_score', 'fake_words_score'])

In [8]:
import numpy as np

In [9]:
mask = np.invert(np.in1d(full.columns, train.columns))

In [10]:
full.columns[mask]

Index(['bot_words_score', 'gen_words_score', 'verified'], dtype='object')

In [11]:
full.verified.value_counts()

1    31347
Name: verified, dtype: int64

In [12]:
bots = pd.read_csv('data/bon/bots/users.csv', engine='python', encoding='utf-8-sig')

In [13]:
humans = pd.read_csv('data/bon/humans/users.csv', engine='python', encoding='utf-8-sig')

In [14]:
def find_verfied(user):
    if user.target == 1:
        selection = bots[bots.friends_count==user.friends_count][bots.followers_count == user.followers_count][bots.statuses_count == user.statuses_count][bots.listed_count == user.listed_count]
        if len(selection) == 1:
            return selection.verified
        else:
            return 0
    else:
        selection = humans[humans.friends_count==user.friends_count][humans.followers_count == user.followers_count][humans.statuses_count == user.statuses_count][humans.listed_count == user.listed_count]
        if len(selection) == 1:
            return selection.verified
        else:
            return 0

In [15]:
len(full[full.target==0])

15692

In [16]:
verified = []

for user in full.itertuples():

    if user.target == 1:
        selection = bots[bots.friends_count==user.friends_count][bots.followers_count == user.followers_count]
        if len(selection) == 1:
            if selection.iloc[0].verified == 'True':
                verified.append(1)
            else:
                verified.append(0)
        else:
            verified.append(0)
    else:
        selection = humans[humans.friends_count==user.friends_count][humans.followers_count == user.followers_count]
        if len(selection) == 1:
            if selection.iloc[0].verified == 'True':
                verified.append(1)
            else:
                verified.append(0)
        else:
            verified.append(0)

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [17]:
full['verified'] = verified

In [18]:
full.verified.value_counts()

0    30773
1      574
Name: verified, dtype: int64

In [19]:
len(full.columns)

36

In [20]:
len(train.columns)

34

In [21]:
train = train.drop('user_id', axis=1)

In [22]:
len(train.columns)

33

In [23]:
train['verified'] = 0

In [24]:
from collections import Counter
from sklearn.preprocessing import MinMaxScaler, normalize

tweets = pd.concat([pd.read_csv('data/porn/tweets.csv', encoding='utf-8-sig', sep='\t'),
                    pd.read_csv('data/propaganda/tweets.csv', encoding='utf-8-sig', sep='\t'),
                    pd.read_csv('data/spam/tweets.csv', encoding='utf-8-sig', sep='\t'),
                    pd.read_csv('data/fake_followers/tweets.csv', encoding='utf-8-sig', sep='\t'),
                    pd.read_csv('data/genuine/tweets.csv', encoding='utf-8-sig', sep='\t')
                   ])

/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,11,18,19,20,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  int

In [25]:
bot_words = pd.read_csv('data/bon/bots/filtered_main_words.csv', sep=',')
gen_words = pd.read_csv('data/bon/humans/filtered_main_words.csv', sep=',')

In [26]:
refill = pd.concat([
    train[train.target==0].sample(2000),
    train[train.target==1].sample(2000),
    train[train.target==2].sample(2000),
    train[train.target==3].sample(2000)
])

In [27]:
genuine = pd.read_csv('data/full/rf_preproc_train.csv')

In [28]:
genuine = genuine[genuine.target==4]

In [29]:
genuine_entropy = pd.read_csv('data/bon/full/validation.csv')

In [30]:
entropies = []

for user in genuine.itertuples():

    selection = genuine_entropy[genuine_entropy.friends_count==user.friends_count][genuine_entropy.followers_count == user.followers_count]
    if len(selection) == 1:
        entropies.append(selection.iloc[0].url_entropy)
    else:
        entropies.append(0)

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [31]:
genuine['url_entropy'] = entropies

In [32]:
mask = np.in1d(genuine.columns, train.columns)

In [33]:
genuine = genuine[genuine.columns[mask]]

In [34]:
genuine['verified'] = 0

In [35]:
len(refill), len(genuine)

(8000, 3661)

In [36]:
refill = pd.concat([refill, genuine])

In [37]:
refill = refill.reset_index()

In [38]:
def edit_target(x):
    if x<=3:
        return 1
    else:
        return 0
    
refill['target'] = refill['target'].apply(lambda x: edit_target(x))

In [39]:
multiclass_users = pd.read_csv('data/full/rf_preproc_train.csv')

In [40]:
ids = []

for user in refill.itertuples():

    selection = multiclass_users[multiclass_users.friends_count==user.friends_count][multiclass_users.followers_count == user.followers_count][multiclass_users.statuses_count == user.statuses_count]
    if len(selection) == 1:
        ids.append(selection.iloc[0].user_id)
    else:
        ids.append(0)

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [41]:
refill['user_id'] = ids

In [42]:
refill=refill[refill.user_id != 0]

In [43]:
def compute_bot_not_score(tweets):

    user_score = pd.DataFrame(columns=['bot_words_score', 'gen_words_score'])

    for tweet in tweets:
        # check for words in main_words and compute the scores for each tweet and for each category
        mask = np.in1d(bot_words.word, tweet.split())
        bot_score = bot_words.loc[mask]['score'].values.sum()
        mask = np.in1d(gen_words.word, tweet.split())
        gen_score = gen_words.loc[mask]['score'].values.sum()
        
        user_score = user_score.append(pd.DataFrame({'bot_words_score': bot_score, 'gen_words_score': gen_score}, index=[0]), ignore_index=True)

    return user_score

In [44]:
def bot_not_score(tweets_df, id):
    
    tweets = tweets_df[tweets_df.user_id.astype(int) == int(id)]['full_text']
    if len(tweets) > 0:
        # sum all the scores of each category
        user_score = compute_bot_not_score(tweets).sum()
        scores = np.divide(user_score,len(tweets))
    else:
        scores = pd.DataFrame({'bot_words_score': 0, 'gen_words_score': 0}, index=[0])
    
    # return the average scores of each user
    return scores

In [45]:
from IPython.display import clear_output

def context_bot_not_score(bot_ids, tweets_df):
        
    score_df = pd.DataFrame(columns=['bot_words_score', 'gen_words_score', 'user_id'])
    i = 0
    for user_id in bot_ids:
        i += 1
        scores = bot_not_score(tweets_df, user_id)
        score_df = score_df.append(scores, ignore_index=True)
        if (i%10 == 0):
            clear_output()
            print(str(i) +  " bots processed")
    
    score_df['user_id'] = bot_ids.values
    
    score_df.reset_index(drop=True, inplace=True)
    #score_df.to_csv('data/' + bot_type + '/context_score.csv', index=False)
    print("done!")
    return score_df

In [46]:
scores = context_bot_not_score(refill.user_id, tweets)

10890 bots processed
done!


In [47]:
refill = refill.merge(scores, on='user_id')

In [50]:
refill.to_csv('data/bon/full/train_with_refill.csv', index=False)

## merge per rimediare alla cazzata

In [34]:
refill = pd.read_csv('data/bon/full/train_with_refill.csv')

In [47]:
refill = refill.drop(columns=['user_id', 'index'])

In [48]:
refill = pd.concat([full,refill])

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


42245